In [ ]:
# import all the necessary libraries

from langchain.document_loaders.csv_loader import CSVLoader
import google.generativeai as genai
import google.ai.generativelanguage as glm
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import os
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
# use google ai studio to get your api key
google_api_key = os.getenv('API_KEY')

In [ ]:
# langchain's way to accessing gemini pro
# google_genai = ChatGoogleGenerativeAI(
#     google_api_key=google_api_key,
#     model='gemini-pro')

model = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=google_api_key,
    temperature=0.1,
    convert_system_message_to_human=True)

# google's way to access gemini pro
# google_genai = genai.configure(api_key=google_api_key)
# llm = genai.GenerativeModel('gemini-pro')

In [ ]:
# llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=google_api_key)

In [ ]:
# embeddings to store vectorized texts
embeddings = GoogleGenerativeAIEmbeddings(
    model='models/embedding-001',
    google_api_key=google_api_key
)

In [ ]:
# load the prompt files
loader = CSVLoader(file_path='prompt_and_response.csv', source_column='prompt')
data = loader.load()

In [ ]:
# intialize the vector db
vector_db = FAISS.from_documents(documents=data, embedding=embeddings)

In [ ]:
# get the retriver object from vector db
retriever = vector_db.as_retriever()
# prompt = 'What is the price of this course?'

In [ ]:
qa = RetrievalQA.from_chain_type(llm=model, retriever=vector_db.as_retriever())

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


chain = RetrievalQA.from_chain_type(llm=model,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    input_key="query",
                                    return_source_documents=True,
                                    chain_type_kwargs=chain_type_kwargs)

In [ ]:
# sample prompt template

prompt_template = """Given the following context and a question, generate an answer based on this context only.
    In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
    If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

    CONTEXT: {context}

    QUESTION: {question}"""